In [1]:
import numpy as np

In [2]:
f = np.array([1,4,0,0])
# f_hat = np.array([5, 249, 766, 522]) # para m = 769
f_hat = np.array([0, 4, 2, 3]) # para m=5

alpha = 2
m = 5
# alpha = 11
# m = 769
N     = 4
# N_inv = 4

In [3]:
## From https://medium.com/geekculture/euclidean-algorithm-using-python-dc7785bb674a
def extended_euclidean(a, b):
    if b == 0:
        gcd, s, t = a, 1, 0
        return (gcd, s, t)  
    else:    
        s2, t2, s1, t1 = 1, 0, 0, 1   
        while b > 0:      
            q= a // b      
            r, s, t = (a - b * q),(s2 - q * s1),( t2 - q * t1)      
            a,b,s2,t2,s1,t1=b,r,s1,t1,s,t    
        gcd,s,t=a,s2,t2    
        return (gcd,s,t)

In [4]:
def get_last_positive_congruence_classes(a,m):
    if a>0: return a
    else:
        k = 1
        while a<0:
            a = a + k*m
            k+=1
        return a

In [5]:
_,x,_ = extended_euclidean(N,m)
N_inv = get_last_positive_congruence_classes(x,m)

Proof #NTT

In [6]:
f_h = []
N = len(f)

for n in range(0, N):
    sum_k = 0

    for k in range(0, N):
            sum_k += f[k] * (alpha**(n*k))
    f_h.append(sum_k % m)

f_h

[0, 4, 2, 3]

Proof # INTT

In [7]:
f_n = []
N = len(f)

for k in range(0, N):
    sum_k = 0

    for n in range(0, N):
            sum_k += f_hat[n] * (alpha** (-(n*k)%N)) ## Change 1
    f_n.append(sum_k % m)

np.mod(N_inv*np.array(f_n), m) ## Change 2

array([1, 4, 0, 0])

Proof #1
$$\hat{f}^{\prime}_{n} = \sum^{N-1}_{k=1}{\underbrace{(kf_{k})}_\text{from (\ref{eq:prop:1b})} \alpha^{n(k-1)}} \mod{m}, \quad n=0, \ldots, N-1$$

In [8]:
for n in range(N):
    sum_o = 0
    for k in range(1,N):
        sum_o += np.mod(k * f[k] * np.power(alpha, n*(k-1)), m)
    print(f'n={n} f_hat_p={sum_o}')

n=0 f_hat_p=4
n=1 f_hat_p=4
n=2 f_hat_p=4
n=3 f_hat_p=4


Proof #2

$$\hat{f}^{\prime}_{n} = \sum^{N-1}_{k=1}{k\underbrace{ \left(N^{-1} \sum_{l=0}^{N-1} \hat{f}_{l} \alpha^{-l k}\right) }_{f_{k}\text{from (\ref{eq:ntt:2b})}} \alpha^{n(k-1)}} \mod{m}, \quad n=0, \ldots, N-1$$

In [9]:
for n in range(N):    
    sum_k = 0
    for k in range(1,N):
        
        sum_l = 0
        for l in range(N):
            sum_l += f_hat[l]*np.power(alpha, -l*k%N ) ## change 1
        sum_l *= N_inv ## change 2
        
        sum_k += k*sum_l*np.power(alpha, n*(k-1))
    sum_k = np.mod(sum_k, m)

    print(f'n={n} f_hat_p={sum_k}')

n=0 f_hat_p=4
n=1 f_hat_p=4
n=2 f_hat_p=4
n=3 f_hat_p=4


Proof #3

$$\hat{f}^{\prime}_{n} = N^{-1}\sum_{l=0}^{N-1} \hat{f}_{l}\sum^{N-1}_{k=1} k \alpha^{k(n-l)}\alpha^{-n} \mod{m}, \quad n=0, \ldots, N-1$$

In [10]:
for n in range(N):    
    
    sum_l = 0
    for l in range(N):
        
        sum_k = 0
        for k in range(1,N):
            sum_k += k*np.power(alpha, (k*(n-l))%N ) ## change 1

        sum_l += f_hat[l]*sum_k
    
    sum_l = N_inv*sum_l*np.power(alpha, -n%N) ## change 2 and 3
    sum_l = np.mod(sum_l, m)

    print(f'n={n} f_hat_p={sum_l}')

n=0 f_hat_p=4
n=1 f_hat_p=4
n=2 f_hat_p=4
n=3 f_hat_p=4


Proof #4

$$\hat{f}^{\prime}_{n} =N^{-1}\sum_{l=0}^{N-1} \hat{f}_{l}\alpha^{-n}  \sum^{N-1}_{k=1} k \alpha^{k(n-l)}  \mod{m}, \quad n=0, \ldots, N-1$$

In [11]:
for n in range(N):    
    
    sum_l = 0
    for l in range(N):
        
        sum_k = 0
        for k in range(1,N):
            sum_k += k*np.power(alpha, (k*(n-l))%N ) ## change 1
        
        print(np.mod(sum_k,m))

        sum_l += f_hat[l]*sum_k
    
    sum_l = N_inv*np.power(alpha, -n%N)*sum_l ##change 2 and 3
    
    sum_l = np.mod(sum_l, m)

    print(f'n={n} f_hat_p={sum_l}')

1
2
3
4
n=0 f_hat_p=4
4
1
2
3
n=1 f_hat_p=4
3
4
1
2
n=2 f_hat_p=4
2
3
4
1
n=3 f_hat_p=4


Proof #5

\begin{equation*}
\sum^{N-1}_{k=1} k \alpha^{k(n-l)} = \begin{cases}
  2^{-1}\left(N-1\right)N  &\text{ if } l=n  \\
  \frac{N}{\left(\alpha^{n-l} -1\right)} &\text{ if } l\neq n,
\end{cases}
\end{equation*}

In [20]:
_, x,_ = extended_euclidean(2, m)
inv_two = get_last_positive_congruence_classes(x, m)

In [21]:
inv_two

3

In [22]:
m

5

In [23]:
np.x

-2

In [13]:
## Left 
for n in range(N):
    for l in range(N):
        sum_k=0
        for k in range(1,N):
            sum_k += k* np.power(alpha, (k*(n-l))%N )

        print(f'l={l},k={n}, sum= {np.mod(sum_k,m)}')
    print('')

l=0,k=0, sum= 1
l=1,k=0, sum= 2
l=2,k=0, sum= 3
l=3,k=0, sum= 4

l=0,k=1, sum= 4
l=1,k=1, sum= 1
l=2,k=1, sum= 2
l=3,k=1, sum= 3

l=0,k=2, sum= 3
l=1,k=2, sum= 4
l=2,k=2, sum= 1
l=3,k=2, sum= 2

l=0,k=3, sum= 2
l=1,k=3, sum= 3
l=2,k=3, sum= 4
l=3,k=3, sum= 1



In [14]:
## Right
for n in range(N):
    for l in range(N):
        if l==n:
            sum_k= inv_two*(N-1)*N
        else:
            pow_a = np.mod((n-l), N) ## only positive exponents
            a = np.power(alpha, pow_a)
            
            if np.gcd(a-1,m)!=1: raise Exception('They are NOT coprimes !!')
            _, x,_ = extended_euclidean(a-1, m)
            a_inv = get_last_positive_congruence_classes(x, m)
            
            sum_k = N * a_inv


        print(f'l={l},l={n}, sum= {np.mod(sum_k,m)}')
    print('')

l=0,l=0, sum= 1
l=1,l=0, sum= 2
l=2,l=0, sum= 3
l=3,l=0, sum= 4

l=0,l=1, sum= 4
l=1,l=1, sum= 1
l=2,l=1, sum= 2
l=3,l=1, sum= 3

l=0,l=2, sum= 3
l=1,l=2, sum= 4
l=2,l=2, sum= 1
l=3,l=2, sum= 2

l=0,l=3, sum= 2
l=1,l=3, sum= 3
l=2,l=3, sum= 4
l=3,l=3, sum= 1



Building the T_prime matrix

In [15]:
l_l = []
for n in range(N):
    n_l = []
    for l in range(N):
        if l==n:
            t_l_n= inv_two*(N-1)*np.power(alpha,-n%N)
        else:
            pow_a = np.mod((n-l), N) ## only positive exponents
            a = np.power(alpha, pow_a)
            
            if np.gcd(a-1,m)!=1: raise Exception('They are NOT coprimes !!')
            _, x,_ = extended_euclidean(a-1, m)
            a_inv = get_last_positive_congruence_classes(x, m)
            
            t_l_n = np.power(alpha, -n%N)*a_inv

        n_l.append(t_l_n)
        # print(f'l={l},l={n}, sum= {t_l_n}')
    l_l.append(n_l)

In [16]:
T_prime = np.array(l_l)
f_h = np.array(f_h)

np.mod(np.dot(T_prime, f_h),m)

array([4, 4, 4, 4])